In [1]:
# first step is to load the documents
from langchain_community.document_loaders import WebBaseLoader
def load_documents(url):
    loader=WebBaseLoader(url)
    loader.requests_per_second=1
    return loader.load()


user_input=input("Enter the URL of the website you want to scrape: ")
documents=load_documents(user_input)
for i,doc in enumerate(documents):
    print(f"Document {i+1}: {doc}")
    

USER_AGENT environment variable not set, consider setting it to identify your requests.


Document 1: page_content='The 7 Habits of Highly Effective People | by Mental Garden | Jan, 2025 | MediumOpen in appSign upSign inWriteSign upSign inThe 7 Habits of Highly Effective PeopleSmall changes, huge resultsMental Garden·Follow7 min read·Jan 26, 2025--ListenShareAlmost all of us know that we could improve in aspects of our lives, both personally and professionally. But here’s the problem: we don’t know where to start or how to do it.Stephen Covey’s book The 7 Habits of Highly Effective People provides a clear and practical answer to overcome just such a blockage.The idea of the book is simple: to maximize the potential of whoever reads it. Covey analyzes patterns observed in great personalities from various fields and translates them into habits that we can all adopt. By the end of reading this text, you will understand how to integrate these principles into your daily life to go one step further in your development.These are the 7 key habits.Habit 1: Being proactive, the power

In [2]:
#second is to split the text
from langchain_text_splitters import RecursiveCharacterTextSplitter # Changed import statement
def split_file(loaded_file):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=20)
    return text_splitter.split_documents(loaded_file)


splited_file=split_file(documents)
splited_file

[Document(metadata={'source': 'https://medium.com/@mentalgarden/the-7-habits-of-highly-effective-people-9ac054ceb590', 'title': 'The 7 Habits of Highly Effective People | by Mental Garden | Jan, 2025 | Medium', 'description': 'Almost all of us know that we could improve in aspects of our lives, both personally and professionally. But here’s the problem: we don’t know where to start or how to do it. Stephen Covey’s book The…', 'language': 'en'}, page_content='The 7 Habits of Highly Effective People | by Mental Garden | Jan, 2025 | MediumOpen in appSign upSign inWriteSign upSign inThe 7 Habits of Highly Effective PeopleSmall changes, huge resultsMental Garden·Follow7 min read·Jan 26, 2025--ListenShareAlmost all of us know that we could improve in aspects of our lives, both personally and professionally. But here’s the problem: we don’t know where to'),
 Document(metadata={'source': 'https://medium.com/@mentalgarden/the-7-habits-of-highly-effective-people-9ac054ceb590', 'title': 'The 7 Ha

In [2]:
from dotenv import load_dotenv 
import os
load_dotenv()
key1=os.environ['chabi1']
key2=os.environ['chabi2']



In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=key1)
vector = embeddings.embed_query("saram")
len(vector)

c:\Users\saram\OneDrive\Desktop\virtualenv\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


768

In [3]:
from langchain_qdrant import QdrantVectorStore
from dotenv import load_dotenv 
import os
load_dotenv()
os.environ['chabi2'] = key2
try:

  url="https://330583f3-8815-45e6-8438-cc963eee1961.europe-west3-0.gcp.cloud.qdrant.io"
  api_key=key2
  qdrant = QdrantVectorStore.from_documents(
    splited_file,
    embeddings,
    url=url,
    
    api_key=api_key,
    collection_name="url-rag-project",
    timeout=120
)
  print("Qdrant collection created--------------->")

except Exception as e:
    print("some network error occured")
    print("Qdrant collection not created--------------->")


some network error occured
Qdrant collection not created--------------->


In [6]:
query=input("what u want to search before chaining??")

results = qdrant.similarity_search(
    query, k=4
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

GoogleGenerativeAIError: Error embedding content: 504 Deadline Exceeded

In [ ]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
try:
# Initialize LLM with the correct model
 llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=key1)

# Convert Qdrant into a retriever
 retriever = qdrant.as_retriever()

# Create the RetrievalQA chain using "stuff" for simple concatenation of documents 
 qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
 )


 while True:
    # Query the chain
    query = input("Enter the prompt you want to search: ")

    # Run the chain with the user's input
    response = qa_chain.run(query)

    # Modify the loop to print unique responses
    if isinstance(response, list):
        for i, res in enumerate(response):
            print(f"MODEL RSP{i+1}: {res}")
    else:
        print(f"MODEL RSP1: {response}")
except Exception as e:
   print("try again network issue.....")
   print("change ur network connectivity.....")

MODEL RSP1: Habit 2 is about beginning with the end in mind.  Its purpose is to ensure that everything you do aligns with your desired outcomes by identifying your values and clarifying your objectives.
MODEL RSP1: Habit 6 is Synergize.  It's about teamwork and collaboration, where the combined effort produces results greater than the sum of individual contributions (1 + 1 = 3, 30, or even 300).  It emphasizes leveraging differences and creating an environment where diverse ideas are welcome.
